# Main compound analisis after compound dicovery step

Requirements: 
- Exported compounds database in excel format
- Exported Chemspider results database in excel format


In [ ]:
import pandas as pd
import chemspi_web_db as chemsearch
#from chemsearch import ChemspiWebDB
from chemspi_local_db import ChemspiLocalDB
import compound_analisis_utils as utils

In [ ]:
# Input
folder = "D:/SharedLAN/plasma_positive_tables/tests/"
compounds_database_file = folder + "positive_plasma_melanoma_triplicate_S7.xlsx"
chemspider_results_db_file = folder + "positive_plasma_melanoma_triplicate_S7_CSID.xlsx"

export_file = folder + "positive_plasma_melanoma_triplicate_S7"

In [ ]:

#web_api_key = "6aAKBdgMGALeExc41btfo6GC48wQSSvH"


# init resources and databases
compounds_table = pd.read_excel(compounds_database_file)
#chempider_local_db = ChemspiLocalDB(chemspider_results_db_file)
#chempider_web_db = ChemspiWebDB(web_api_key)

# remove not needed columns
compounds_table.drop(columns=['Checked'], inplace=True)


print("Sucess reading databases")
#display(compounds_table)

## Step 1: Removing duplicated entries
- Sort by Name
- Sort by area
- Sort by retention time
- Select from duplicated names: one with highest area then retention time

In [ ]:

compounds_table = utils.remove_duplicated_entries(compounds_table)
print("Done")
compounds_table.to_excel(export_file + "_step1.xlsx")
#display(compounds_table)

# Step 2: Find CSId from chemspi databases
- Try local first
- Try online otherwise

In [ ]:
#compounds_table = pd.read_excel(export_file + "_step1.xlsx")
csid_values = []
iupac_values = []

num_rows = compounds_table.shape[0]
row_count = 0
for row_index, row_data in compounds_table.iterrows() :    
    row_count += 1
    compound_name = row_data.Name.lower()
    compound_mass = row_data['Molecular Weight']
    
    print('({:d}/{:d}) searching compound info '.format(row_count, num_rows) + compound_name)
    
    http_compounds = chemsearch.http_find_compounds_by_name_mass(compound_name, compound_mass)

    if len(http_compounds) > 1 :
        csids = ""
        for c in http_compounds:
            csids += str(c.csid) + ";";
        #print("Found csids: ", csids)
        csid_values.append(csids)
        iupac_values.append(http_compounds[0].iupac_name)
    elif len(http_compounds) == 1 :
        csid_values.append(http_compounds[0].csid)
        iupac_values.append(http_compounds[0].iupac_name)
    else:
        #print("Not Found")
        csid_values.append('ND')
        iupac_values.append('ND')

        
compounds_table.insert(0, 'CSID', csid_values)
compounds_table.insert(1, 'IUPAC Name', iupac_values)
display(compounds_table)
compounds_table.to_excel(export_file + "_step2.xlsx")

# Step 3: Find External references from chemspi web search

In [ ]:
#compounds_table = pd.read_excel(export_file + "_step2.xlsx")
external_databases = ['Human Metabolome Database', 'KEGG', 'LipidMAPS']
external_db_info = dict()
for db in external_databases:
    external_db_info[db] = []
        
num_rows = compounds_table.shape[0]
row_count = 0
for row_index, row_data in compounds_table.iterrows() :
    # search for external references from known databases
    row_count += 1
    print('({:d}/{:d}) Finding external references for '.format(row_count, num_rows) + row_data.Name)
    external_databases_result = {}
    for db in external_databases:
        external_databases_result[db] = []
    
    for csid in utils.parse_generated_CSID(row_data.CSID):
        print("CSID: ", csid)
        external_refs = chemsearch.http_find_compound_external_sources_ids(csid, external_databases)
        display(external_refs)
        
        #if len(warnings) > 0 :
        #    display(warnings)
        
        for db, values in external_refs.items():
            external_databases_result[db].extend(values)
    
    for db, values in external_databases_result.items():
        str_result = ""
        for v in values :
            str_result += str(v) + ";"
        external_db_info[db].append(str_result)
        
insert_pos = 1
for db_name, value in external_db_info.items():
    compounds_table.insert(insert_pos, db_name, value)
    insert_pos += 1

display(compounds_table)
compounds_table.to_excel(export_file + "_step3.xlsx")
print("Done!")